In [1]:
from dataloader import *
from  config import *
from data_processor import preprocess_data

In [2]:
data = DataLoader(config_mixed_coco14_flickr8k_vgg19_glove)

Loading dataset
Loading train dataset
Loading images splits
Images splits loaded
Number of train images:  113287
Number of test images:  5000
Loading all captions
All captions loaded
Nuber of all captions:  123287
Loading captions splits
Captions splits loaded
Number of train captions:  113287
Number of test test:  5000
Loading test dataset
Loading images splits
Images splits loaded
Number of train images:  6000
Number of test images:  1000
Loading all captions
All captions loaded
Nuber of all captions:  8092
Loading captions splits
Captions splits loaded
Number of train captions:  6000
Number of test test:  1000


In [3]:
data=preprocess_data(data)

Number of train images:  113287
Number of test images:  1000
Number of train captions:  113287
Number of test captions:  1000
Descriptions cleaned.
[['a', 'woman', 'wearing', 'a', 'net', 'on', 'her', 'head', 'cutting', 'a', 'cake'], ['a', 'woman', 'cutting', 'a', 'large', 'white', 'sheet', 'cake'], ['a', 'woman', 'wearing', 'a', 'hair', 'net', 'cutting', 'a', 'large', 'sheet', 'cake'], ['there', 'is', 'a', 'woman', 'that', 'is', 'cutting', 'a', 'white', 'cake'], ['a', 'woman', 'marking', 'a', 'cake', 'with', 'the', 'back', 'of', 'a', 'chefs', 'knife']]
Descriptions wraped into start and stop words.
['START a woman wearing a net on her head cutting a cake STOP', 'START a woman cutting a large white sheet cake STOP', 'START a woman wearing a hair net cutting a large sheet cake STOP', 'START there is a woman that is cutting a white cake STOP', 'START a woman marking a cake with the back of a chefs knife STOP']
Used: Vgg19
Processed:
100
Processed:
200
Processed:
300
Processed:
400
Process

Processed:
41100
Processed:
41200
Processed:
41300
Processed:
41400
Processed:
41500
Processed:
41600
Processed:
41700
Processed:
41800
Processed:
41900
Processed:
42000
Processed:
42100
Processed:
42200
Processed:
42300
Processed:
42400
Processed:
42500
Processed:
42600
Processed:
42700
Processed:
42800
Processed:
42900
Processed:
43000
Processed:
43100
Processed:
43200
Processed:
43300
Processed:
43400
Processed:
43500
Processed:
43600
Processed:
43700
Processed:
43800
Processed:
43900
Processed:
44000
Processed:
44100
Processed:
44200
Processed:
44300
Processed:
44400
Processed:
44500
Processed:
44600
Processed:
44700
Processed:
44800
Processed:
44900
Processed:
45000
Processed:
45100
Processed:
45200
Processed:
45300
Processed:
45400
Processed:
45500
Processed:
45600
Processed:
45700
Processed:
45800
Processed:
45900
Processed:
46000
Processed:
46100
Processed:
46200
Processed:
46300
Processed:
46400
Processed:
46500
Processed:
46600
Processed:
46700
Processed:
46800
Processed:
469

Processed:
89300
Processed:
89400
Processed:
89500
Processed:
89600
Processed:
89700
Processed:
89800
Processed:
89900
Processed:
90000
Processed:
90100
Processed:
90200
Processed:
90300
Processed:
90400
Processed:
90500
Processed:
90600
Processed:
90700
Processed:
90800
Processed:
90900
Processed:
91000
Processed:
91100
Processed:
91200
Processed:
91300
Processed:
91400
Processed:
91500
Processed:
91600
Processed:
91700
Processed:
91800
Processed:
91900
Processed:
92000
Processed:
92100
Processed:
92200
Processed:
92300
Processed:
92400
Processed:
92500
Processed:
92600
Processed:
92700
Processed:
92800
Processed:
92900
Processed:
93000
Processed:
93100
Processed:
93200
Processed:
93300
Processed:
93400
Processed:
93500
Processed:
93600
Processed:
93700
Processed:
93800
Processed:
93900
Processed:
94000
Processed:
94100
Processed:
94200
Processed:
94300
Processed:
94400
Processed:
94500
Processed:
94600
Processed:
94700
Processed:
94800
Processed:
94900
Processed:
95000
Processed:
951

2196it [00:00, 21956.51it/s]

preprocessed words 27984 -> 7292
Vocab size:  7293
Glove used


400000it [00:12, 32792.16it/s]


Found 400000 word vectors.
(7293, 199)


In [4]:
from keras.layers import LSTM, Embedding, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.layers.merge import add
from keras.models import Model
from keras import Input
from keras import callbacks
from eval_utils import calculate_results, prepare_for_evaluation
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from numpy import array
from eval_utils import generate_report

In [5]:
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch, vocab_size):
    """
    Data generator, that serves the data to the model during training


    Parameters
    ----------
    descriptions: str
        Dictionary where key is image id and value is a list of wraped in start and stop captions, lemmatized, without punctuation descriptions
    photos
        Dictionary with encoded images(vector of image features extracted by specified image feature extractor
         fe. Inception), identified by image id
    wordtoix
        Dictionary with keys-words , values -id of word
    max_length
        Max number of words in caption on dataset
    num_photos_per_batch: int
    vocab_size: int
    Returns
    -------
    """
    X1, X2, y = list(), list(), list()
    n = 0
    #iterujemy po opisach doobrazu
    # kazdy opis zamieniamy na wektor liczb za pomoca slownika wordtoix
    # tworzymy mase par (zdjęcie + slowa) i sekwencja wyjsciowa. Czyli na bazie czeci zdania i zdjecia przewidujemy reszte zdania
    while 1:
        for image_id, desc_list in descriptions.items():
            n += 1
            # retrieve the photo feature from the dictionary
            photo = photos[image_id]
            for desc in desc_list:
                # encode the sentence by translating it to the number representation,
                # with the dictionary of words created in in the previous stage
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n == num_photos_per_batch:
                yield ([array(X1), array(X2)], array(y))
                X1, X2, y = list(), list(), list()
                n = 0

class ModelImpl:
    def __init__(self, data):
        self.data=data
        if data.configuration['images_processor'] == 'vgg16' or data.configuration['images_processor'] == 'vgg19':
            inputs1= Input(shape=(4096,))
        else:
            inputs1 = Input(shape=(2048,))
        fe1 = Dropout(0.5)(inputs1)
        fe2 = Dense(256, activation='relu')(fe1)
        inputs2 = Input(shape=(self.data.max_length,))
        # The Embedding layer can be understood as a lookup table that maps from integer
        # indices (which stand for specific words) to dense vectors (their embeddings).
        if data.configuration["text_processor"] == "fastText":
            se1 = Embedding(self.data.vocab_size, fastText[self.data.language]["embedings_dim"], mask_zero=True)(inputs2)
        elif data.configuration["text_processor"] == "word2Vec":
            se1 = Embedding(self.data.vocab_size, word2Vec[self.data.language]["embedings_dim"], mask_zero=True)(inputs2)
        else:
            se1 = Embedding(self.data.vocab_size, glove[self.data.language]["embedings_dim"], mask_zero=True)(inputs2)
        se2 = Dropout(0.5)(se1)
        se3 = LSTM(256)(se2)
        decoder1 = add([fe2, se3])
        decoder2 = Dense(256, activation='relu')(decoder1)
        outputs = Dense(self.data.vocab_size, activation='softmax')(decoder2)
        self.model = Model(inputs=[inputs1, inputs2], outputs=outputs)
        self.model.summary()
        self.model.layers[2]

        self.model.layers[2].set_weights([self.data.embedding_matrix])
        self.model.layers[2].trainable = False

        self.model.compile(loss='categorical_crossentropy', optimizer=self.optimizer())
        self.setup()

    def optimizer(self):
        return Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    def setup(self):
        # model.optimizer.lr = 0.001
        self.epochs = 100
        self.number_pics_per_bath = 3
        self.steps = len(self.data.train_captions_wrapped) // self.number_pics_per_bath

    def train(self):
        model_weights_path="./" + self.data.configuration["data_name"] + self.data.configuration["model_save_dir"]
        if self.data.configuration["train_model"]:
            callback = callbacks.EarlyStopping(monitor='loss', min_delta=0.001, patience=3)
            generator = data_generator(self.data.train_captions_wrapped,
                                       self.data.encoded_images_train,
                                       self.data.wordtoix,
                                       self.data.max_length,
                                       self.number_pics_per_bath,
                                       self.data.vocab_size)
            self.model.fit(generator, epochs=self.epochs,
                           steps_per_epoch=self.steps,
                           callbacks=[callback],
                           verbose=1)
            if self.data.configuration["save_model"]:
                writepath = model_weights_path+ "/"+'model' + '.h5'
                self.model.save(writepath)
                self.model.save_weights(model_weights_path
                                        + self.data.configuration["model_save_path"])
        else:
            self.model.load_weights(model_weights_path
                                        +self.data.configuration["model_save_path"])

    def evaluate(self):
        expected, results = prepare_for_evaluation(self.data.encoded_images_test, self.data.test_captions_mapping,
                                                   self.data.wordtoix, self.data.ixtoword, self.data.max_length,
                                                   self.model, self.data.configuration["images_processor"])
        out = calculate_results(expected, results, self.data.configuration)
        print(out)

model=ModelImpl(data)


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 51)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 4096)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 51, 199)      1451307     input_3[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 4096)         0           input_2[0][0]                    
____________________________________________________________________________________________

In [6]:
model.train()

OSError: Unable to open file (unable to open file: name = './mixed_coco14_flickr8k_inception_glove/model_weights/model_Base_3_Batch_Komninos.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
model.evaluate()

In [ ]:
generate_report(general["results_directory"])
